In [1]:
import pandas as pd
import os
import tarfile
import gzip

In [2]:
PATH_BA = '../Data/BeerAdvocate_CSV.tar.gz'
PATH_RB = '../Data/RateBeer_CSV.tar.gz'
folder_BA = tarfile.open(PATH_BA)
folder_RB = tarfile.open(PATH_RB)

In [3]:
# Extracting the files for BeerAdvocate
df_beers_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/beers.csv')))
df_users_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/users.csv')))
df_reviews_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/reviews.csv')))
df_breweries_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/breweries.csv')))

# Extracting the files for RateBeer
df_beers_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/beers.csv')))
df_users_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/users.csv')))
df_reviews_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/reviews.csv')))
df_breweries_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/breweries.csv')))


In [4]:
user_nat_RB=pd.DataFrame()
user_nat_RB['user_name']=df_users_RB['user_name']
user_nat_RB['location_user']=df_users_RB['location']
new_reviews_RB=pd.merge(df_reviews_RB, user_nat_RB,  how='inner', on='user_name')

beers_nat_RB=pd.DataFrame()
beers_nat_RB['beer_id']=df_breweries_RB['id']
beers_nat_RB['beers_location']=df_breweries_RB['location']
new_reviews_RB['beer_id']=new_reviews_RB['beer_id'].apply(lambda x: int(x))
df_RB= pd.merge(new_reviews_RB, beers_nat_RB, how='inner', on='beer_id')
df_RB=df_RB.dropna(subset=['appearance', 'aroma','palate', 'taste', 'overall', 'rating', 'location_user', 'beers_location'])

In [5]:
user_nat_BA=pd.DataFrame()
user_nat_BA['user_name']=df_users_BA['user_name']
user_nat_BA['location_user']=df_users_BA['location']
new_reviews_BA=pd.merge(df_reviews_BA, user_nat_BA,  how='inner', on='user_name')

beers_nat_BA=pd.DataFrame()
beers_nat_BA['beer_id']=df_breweries_BA['id']
beers_nat_BA['beers_location']=df_breweries_BA['location']
new_reviews_BA['beer_id']=new_reviews_BA['beer_id'].apply(lambda x: int(x))
df_BA = pd.merge(new_reviews_BA, beers_nat_BA, how='inner', on='beer_id')
df_BA=df_BA.dropna(subset=['appearance', 'aroma','palate', 'taste', 'overall', 'rating', 'location_user', 'beers_location'])

In [6]:
US_states = pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-2:US')[0] 
US_states['Subdivision name (en)'] = US_states['Subdivision name (en)'].apply(lambda x: 'United States, ' + x) 
US_states['Code'] = US_states['Code'].apply(lambda x: x[3:]) 
US_states.drop(columns=['Subdivision category'], inplace=True) 
 
US_states_user=US_states.rename(columns={'Subdivision name (en)':'location_user', 'Code':'US_Code_User'}) 
US_states_beer=US_states.rename(columns={'Subdivision name (en)':'beers_location', 'Code':'US_Code_Beer'}) 
US_states_beer 
df_BA=pd.merge(US_states_beer, df_BA, how='outer', on='beers_location') 
df_BA=pd.merge(US_states_user, df_BA, how='outer', on='location_user') 

df_RB=pd.merge(US_states_beer, df_RB, how='outer', on='beers_location')
df_RB=pd.merge(US_states_user, df_RB, how='outer', on='location_user')
 
def keep_United_States_if_in_the_string(x): 
    if 'United States' in x: 
        return 'United States' 
    else: 
        return x 
 


df_BA['beers_location']=df_BA['beers_location'].apply(lambda x: str(x)) 
df_BA['beers_location']=df_BA['beers_location'].apply(lambda x: keep_United_States_if_in_the_string(x)) 
df_BA['location_user']=df_BA['location_user'].apply(lambda x: str(x)) 
df_BA['location_user']=df_BA['location_user'].apply(lambda x: keep_United_States_if_in_the_string(x)) 

df_RB['beers_location']=df_RB['beers_location'].apply(lambda x: str(x))
df_RB['beers_location']=df_RB['beers_location'].apply(lambda x: keep_United_States_if_in_the_string(x))
df_RB['location_user']=df_RB['location_user'].apply(lambda x: str(x))
df_RB['location_user']=df_RB['location_user'].apply(lambda x: keep_United_States_if_in_the_string(x))
df_BA

,US_Code_User,location_user,US_Code_Beer,beers_location,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,text
0,AL,United States,AL,United States,Tucher Dunkles Hefe Weizen,370.0,Brauerei Tucher Brau,129.0,Dunkelweizen,5.35,1.315476e+09,JayS2629,jays2629.517433,4.50,4.00,4.5,4.00,4.0,4.08,Pours a hazy dark brown with huge creamy white...
1,AL,United States,AL,United States,Tucher Dunkles Hefe Weizen,370.0,Brauerei Tucher Brau,129.0,Dunkelweizen,5.35,1.139483e+09,EvilKeith,evilkeith.53115,4.00,4.00,4.0,4.00,4.0,4.00,"Poured a beautiful deep, dark amber with the m..."
2,AL,United States,AL,United States,Tucher Dunkles Hefe Weizen,370.0,Brauerei Tucher Brau,129.0,Dunkelweizen,5.35,1.345630e+09,Reaper16,reaper16.254804,4.00,3.50,3.0,3.00,3.0,3.18,500mL bottle served into a weizen glassPours a...
3,AL,United States,AL,United States,Tucher Dunkles Hefe Weizen,370.0,Brauerei Tucher Brau,129.0,Dunkelweizen,5.35,1.174388e+09,GPHarris,gpharris.70907,4.00,3.50,4.0,4.00,4.0,3.88,a - poured a four finger creamy off-white and ...
4,AL,United States,AL,United States,Tucher Dunkles Hefe Weizen,370.0,Brauerei Tucher Brau,129.0,Dunkelweizen,5.35,1.126174e+09,Bonhoeffer,bonhoeffer.3756,4.50,4.00,4.0,3.50,3.5,3.73,My first dunkel weizen. I was shocked when i r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721412,NaN,Uruguay,NaN,England,Patricia,47384.0,Fabricas Nacionales De Cerveza,18389.0,American Adjunct Lager,5.00,1.422270e+09,Hidalgo55,hidalgo55.933997,3.75,3.75,4.0,4.00,4.0,3.93,"Patricia is a classic Uruguayan beer, and alth..."
721413,NaN,Uruguay,NaN,England,Patricia,47384.0,Fabricas Nacionales De Cerveza,18389.0,American Adjunct Lager,5.00,1.288001e+09,HombreUruguay,hombreuruguay.517961,4.00,3.50,4.0,4.00,4.0,3.88,"Patricia is a Classic Uruguayan beer, and it i..."
721414,NaN,Uruguay,NaN,Brazil,Zillertal,48696.0,Fabricas Nacionales De Cerveza,18389.0,Euro Pale Lager,5.50,1.422270e+09,Hidalgo55,hidalgo55.933997,4.00,3.75,4.0,4.25,4.0,4.04,This is one of my favorite beers from Uruguay....
721415,NaN,Estonia,NaN,Russia,Weltenburger Kloster Barock Dunkel,4920.0,Klosterbrauerei Weltenburg,1315.0,Munich Dunkel Lager,4.70,1.459591e+09,RoverDnB,roverdnb.1117435,5.00,5.00,5.0,5.00,5.0,5.00,5/5The best bear i have ever tasted!Actually f...


In [7]:
count_contry_users=df_RB.groupby(['location_user']).count()['beer_name'].to_frame() 
count_contry_users.reset_index(inplace=True) 
count_contry_users.iloc[0,1]

import plotly.express as px 
 
fig = px.choropleth(count_contry_users, 
                    locations='location_user',  
                    locationmode='country names',  
                    scope="world", 
                    color='beer_name',  
                     
                    ) 
fig.show()